In [1]:
import pandas as pd
import numpy as np

import random
from scipy.stats import truncnorm
import uuid

# Requisito 1

In [3]:
d1 = {
    "ds": "dataset1",
    "columns": [
        {
            "name": "area",
            "type": "category",
            "values": ["TI", "FIN", "HR"]
        },
        {
            "name": "id",
            "type": "unique"
        }
    ],
    "random": False
}

In [4]:
d1

{'ds': 'dataset1',
 'columns': [{'name': 'area',
   'type': 'category',
   'values': ['TI', 'FIN', 'HR']},
  {'name': 'id', 'type': 'unique'}],
 'random': False}

In [5]:
def generate_unique_id(n):
    return [str(uuid.uuid4()).replace('-', '')[:16] for _ in range(n)]

# Determinar el número máximo de filas
max_rows = max(len(column.get("values", [])) for column in d1["columns"] if "values" in column)

# Crear un diccionario para almacenar los datos del DataFrame
data = {}

# Generar los datos para cada columna en la configuración
for column in d1["columns"]:
    column_name = column["name"]
    column_type = column["type"]
    
    if column_type == "category":
        values = column["values"]
        # Repetir los valores para llenar hasta el número máximo de filas
        repeated_values = (values * (max_rows // len(values) + 1))[:max_rows]
        data[column_name] = pd.Categorical(repeated_values)
    elif column_type == "unique":
        data[column_name] = generate_unique_id(max_rows)
    else:
        raise ValueError(f"Tipo de columna no soportado: {column_type}")


# Crear el DataFrame
df = pd.DataFrame(data)

# Imprimir el DataFrame
df

,area,id
0,TI,0e7eb5be6e154d67
1,FIN,c6ea5ce6d28f420a
2,HR,1c996894070d4b11


# Requisito 2

In [7]:
d2 = {
    "ds": "dataset2",
    "columns": [
        {
            "name": "id",
            "type": "unique"
        },
        {
            "name": "area",
            "type": "foreign",
            "values": "dataset1.area"
        },
        {
            "name": "subarea",
            "type": "category",
            "values": ["SA1", "SA2", "SA3", "SA4"]
        }
    ],
    "random": False
}

In [8]:
d2

{'ds': 'dataset2',
 'columns': [{'name': 'id', 'type': 'unique'},
  {'name': 'area', 'type': 'foreign', 'values': 'dataset1.area'},
  {'name': 'subarea',
   'type': 'category',
   'values': ['SA1', 'SA2', 'SA3', 'SA4']}],
 'random': False}

In [9]:
# Verificacion
referenced_dataset = d1  # Referencia al dataset 1

# Verificar que el dataset de referencia exista
if d2["columns"][1]["values"].split('.')[0] != referenced_dataset['ds']:
    raise ValueError(f"Referencia inválida al dataset. Dataset válido: {referenced_dataset['ds']}")

# Verificar que la columna de referencia exista
if d2["columns"][1]["values"].split('.')[1] not in [col['name'] for col in referenced_dataset['columns']]:
    raise ValueError(f"Columna de referencia inválida. Columnas válidas: {[col['name'] for col in referenced_dataset['columns']]}")

In [10]:
# Función encargada de calcular la columna con mayor cantidad de registros
def calc_greater(cols_arr):
    max_values = 0
    
    for col in cols_arr:
        col_type = col['type']

        if col_type == 'foreign':
            continue  # Ignorar, ya que no tiene valores directos

        if col_type == 'date':
            max_values = max(max_values, pd.date_range(start=col['values']['min'], end=col['values']['max']).size)

        if col_type == 'category':
            max_values = max(max_values, len(col['values']))

    return max_values

# Determinar el número máximo de filas basado en la columna con más datos
max_rows = calc_greater(d2["columns"])

In [11]:
data2 = {}

# Selección aleatoria de valores para la columna 'area' del dataset 1
random_areas = [random.choice(referenced_dataset["columns"][0]["values"]) for _ in range(max_rows)]

# Generar los datos para cada columna en la configuración del dataset2
for column in d2["columns"]:
    column_name = column["name"]
    column_type = column["type"]
    
    if column_type == "category":
        values = column["values"]
        # Repetir los valores para llenar hasta el número máximo de filas
        repeated_values = (values * (max_rows // len(values) + 1))[:max_rows]
        data2[column_name] = pd.Categorical(repeated_values)
    elif column_type == "unique":
        data2[column_name] = generate_unique_id(max_rows)
    elif column_type == "foreign":
        # Repetir los valores del dataset1.area para llenar hasta el número máximo de filas
        repeated_values = (random_areas * (max_rows // len(random_areas) + 1))[:max_rows]
        data2[column_name] = pd.Categorical(repeated_values)
    else:
        raise ValueError(f"Tipo de columna no soportado: {column_type}")

# Crear el DataFrame del dataset2
df2 = pd.DataFrame(data2)

# Imprimir el DataFrame del dataset2
df2

,id,area,subarea
0,d849c1d4fdee4988,FIN,SA1
1,329e5a2170e344eb,HR,SA2
2,b9a1ab7e191e4326,HR,SA3
3,dc4d2227ac074ebf,FIN,SA4


# Requisito 3

In [13]:
d3 = {
    "ds": "dataset3",
    "columns": [
        {
            "name": "id",
            "type": "unique"
        },
        {
            "name": "subarea",
            "type": "foreign",
            "values": "dataset2.id"
        },
        {
            "name": "income",
            "type": "numeric",
            "values": {
                "min": 10000,
                "max": 100000
            }
        },
        {
            "name": "goal",
            "type": "numeric",
            "values": {
                "min": 10000,
                "max": 100000,
                "std": 50000,
                "mean": 50000
            }
        }
    ],
    "random": True,
    "random_rows": 1000
}

In [14]:
d3

{'ds': 'dataset3',
 'columns': [{'name': 'id', 'type': 'unique'},
  {'name': 'subarea', 'type': 'foreign', 'values': 'dataset2.id'},
  {'name': 'income',
   'type': 'numeric',
   'values': {'min': 10000, 'max': 100000}},
  {'name': 'goal',
   'type': 'numeric',
   'values': {'min': 10000, 'max': 100000, 'std': 50000, 'mean': 50000}}],
 'random': True,
 'random_rows': 1000}

In [15]:
# Función para generar valores utilizando una distribución normal
def generate_normal(min_val, max_val, size):
    mean = (min_val + max_val) / 2
    std = (max_val - min_val) / 4  # Aproximación: usar 1/4 del rango como desviación estándar
    return np.random.normal(loc=mean, scale=std, size=size).astype(int).clip(min_val, max_val)

# Función para generar valores utilizando una distribución normal truncada
def generate_truncated_normal(mean, std, min_val, max_val, size):
    a, b = (min_val - mean) / std, (max_val - mean) / std
    return truncnorm.rvs(a, b, loc=mean, scale=std, size=size).astype(int)

# Generar el número de filas basado en random_rows
num_rows = d3["random_rows"]

# Crear un diccionario para almacenar los datos del dataset3
data3 = {}

#referenced_dataset = d2  # Referencia al dataset 1
# Selección aleatoria de valores para la columna 'area' del dataset 1
#subarea_ids = [random.choice(referenced_dataset["columns"][1]["values"]) for _ in range(num_rows)]

In [16]:
# Generar valores para cada columna en la configuración de d3
for column in d3["columns"]:
    column_name = column["name"]
    column_type = column["type"]

    if column_type == "unique":
        data3[column_name] = generate_unique_id(num_rows)

    elif column_type == "foreign":
        if column_name == "subarea":
            # Por ejemplo, supongamos que estos son los IDs que vienen de dataset2
            #subarea_ids = [str(uuid.uuid4()).replace('-', '')[:16] for _ in range(max_rows)]
            #data3[column_name] = [random.choice(subarea_ids) for _ in range(num_rows)]
            data3[column_name] = [random.choice(df2["id"]) for _ in range(num_rows)]
        else:
            values = column["values"]
            # Repetir los valores para llenar hasta el número máximo de filas
            repeated_values = (values * (num_rows // len(values) + 1))[:num_rows]
            data3[column_name] = pd.Categorical(repeated_values)

    elif column_type == "numeric":
        values_config = column["values"]
        min_val = values_config["min"]
        max_val = values_config["max"]

        if "mean" in values_config and "std" in values_config:
            mean = values_config["mean"]
            std = values_config["std"]
            data3[column_name] = generate_truncated_normal(mean, std, min_val, max_val, num_rows)
        else:
            data3[column_name] = generate_normal(min_val, max_val, num_rows)

# Crear el DataFrame del dataset3
df3 = pd.DataFrame(data3)

# Imprimir el DataFrame del dataset3
df3

,id,subarea,income,goal
0,5bd43e10a2e344cd,d849c1d4fdee4988,32020,87738
1,1d568db1f22d4989,b9a1ab7e191e4326,61559,11467
2,27d3804538954da2,329e5a2170e344eb,67982,47385
3,1774cb0896534484,dc4d2227ac074ebf,44383,26191
4,4a64e87df6914d4c,d849c1d4fdee4988,32943,59062
...,...,...,...,...
995,9f079784edd84c16,b9a1ab7e191e4326,51203,28284
996,6b7610c6e1654d87,dc4d2227ac074ebf,61160,43540
997,3d4dbb2400ee4dd6,dc4d2227ac074ebf,80691,51518
998,6562bedde149478b,d849c1d4fdee4988,21357,32039


# Requisito 4

In [18]:
# Configuración de d4 y d5
d4 = {
    "ds": "dataset4",
    "columns": [
        {
            "name": "id",
            "type": "unique"
        },
        {
            "name": "department",
            "type": "category",
            "values": ["Sales", "Engineering", "Marketing", "Finance", "HR"]
        },
        {
            "name": "age",
            "type": "numeric",
            "values": {
                "min": 22,
                "max": 65
            }
        },
        {
            "name": "experience",
            "type": "numeric",
            "values": {
                "min": 1,
                "max": 40,
                "mean": 15,
                "std": 10
            }
        }
    ],
    "random": False
}

d5 = {
    "ds": "dataset5",
    "columns": [
        {
            "name": "id",
            "type": "unique"
        },
        {
            "name": "employee_id",
            "type": "foreign",
            "values": "dataset4.id"
        },
        {
            "name": "country",
            "type": "category",
            "values": ["USA", "Canada", "UK", "Germany", "France", "India", "China", "Japan"]
        },
        {
            "name": "status",
            "type": "category",
            "values": ["Full-time", "Part-time", "Contract", "Intern"]
        },
        {
            "name": "salary",
            "type": "numeric",
            "values": {
                "min": 30000,
                "max": 150000
            }
        },
        {
            "name": "bonus",
            "type": "numeric",
            "values": {
                "min": 10000,
                "max": 20000,
                "mean": 50000,
                "std": 30000
            }
        },
        {
            "name": "joining_date",
            "type": "date",
            "values": {
                "min": "2000-01-01",
                "max": "2023-12-31"
            }
        }
    ],
    "random": True,
    "random_rows": 1000
}


In [19]:
d4

{'ds': 'dataset4',
 'columns': [{'name': 'id', 'type': 'unique'},
  {'name': 'department',
   'type': 'category',
   'values': ['Sales', 'Engineering', 'Marketing', 'Finance', 'HR']},
  {'name': 'age', 'type': 'numeric', 'values': {'min': 22, 'max': 65}},
  {'name': 'experience',
   'type': 'numeric',
   'values': {'min': 1, 'max': 40, 'mean': 15, 'std': 10}}],
 'random': False}

In [20]:
d5

{'ds': 'dataset5',
 'columns': [{'name': 'id', 'type': 'unique'},
  {'name': 'employee_id', 'type': 'foreign', 'values': 'dataset4.id'},
  {'name': 'country',
   'type': 'category',
   'values': ['USA',
    'Canada',
    'UK',
    'Germany',
    'France',
    'India',
    'China',
    'Japan']},
  {'name': 'status',
   'type': 'category',
   'values': ['Full-time', 'Part-time', 'Contract', 'Intern']},
  {'name': 'salary',
   'type': 'numeric',
   'values': {'min': 30000, 'max': 150000}},
  {'name': 'bonus',
   'type': 'numeric',
   'values': {'min': 10000, 'max': 20000, 'mean': 50000, 'std': 30000}},
  {'name': 'joining_date',
   'type': 'date',
   'values': {'min': '2000-01-01', 'max': '2023-12-31'}}],
 'random': True,
 'random_rows': 1000}

In [21]:
# Función para generar fechas aleatorias
def get_random_dates(min_date, max_date, n):
    items = pd.date_range(start=min_date, end=max_date, freq='D')
    return random.choices(items, k=n)

In [22]:
# Generar valores de categoría aleatorios
def generate_category_random_values(values, size):
    return np.random.choice(values, size=size, replace=True)

In [23]:
def generate_dataset(config, reference_data=None):
    num_rows = config["random_rows"] if config["random"] else calc_greater(config["columns"])
    data = {}

    for column in config["columns"]:
        column_name = column["name"]
        column_type = column["type"]

        if column_type == "unique":
            data[column_name] = generate_unique_id(num_rows)

        elif column_type == "category":
            values = column["values"]
            if isinstance(values, list):
                repeated_values = (values * (num_rows // len(values) + 1))[:num_rows]
                data[column_name] = pd.Categorical(repeated_values)
            elif isinstance(values, str) and reference_data and values in reference_data:
                ref_values = reference_data[values]
                repeated_values = (ref_values * (num_rows // len(ref_values) + 1))[:num_rows]
                data[column_name] = pd.Categorical(repeated_values)

        elif column_type == "numeric":
            values_config = column["values"]
            min_val = values_config["min"]
            max_val = values_config["max"]

            if "mean" in values_config and "std" in values_config:
                mean = values_config["mean"]
                std = values_config["std"]
                data[column_name] = generate_truncated_normal(mean, std, min_val, max_val, num_rows)
            else:
                data[column_name] = generate_normal(min_val, max_val, num_rows)

        elif column_type == "date":
            start_date = pd.to_datetime(column["values"]["min"])
            end_date = pd.to_datetime(column["values"]["max"])
            data[column_name] = get_random_dates(start_date, end_date, num_rows)

        elif column_type == "foreign":
            if reference_data is not None and column["values"] in reference_data:
                values = reference_data[column["values"]]
                repeated_values = (values * (num_rows // len(values) + 1))[:num_rows]
                data[column_name] = pd.Categorical(repeated_values)

    return pd.DataFrame(data)



# Generar el dataset d4
df4 = generate_dataset(d4)

# Generar el dataset d5 usando los datos del dataset d4
reference_data = {"dataset4.id": df4["id"].tolist()}
df5 = generate_dataset(d5, reference_data)

In [24]:
print("Dataset4:")
df4

Dataset4:


,id,department,age,experience
0,93472f46e1ba4d79,Sales,50,15
1,71d199d9b2c7444a,Engineering,37,18
2,1a1c283f932640ed,Marketing,43,18
3,21eddc7dd0ac48a8,Finance,43,19
4,e60b849c20874ba9,HR,22,1


In [25]:
print("\nDataset5:")
df5


Dataset5:


,id,employee_id,country,status,salary,bonus,joining_date
0,cccd29d8967a477f,93472f46e1ba4d79,USA,Full-time,131131,19830,2022-09-09
1,3a943ac411cd49b0,71d199d9b2c7444a,Canada,Part-time,79243,19489,2009-06-16
2,2a9e512221cf4968,1a1c283f932640ed,UK,Contract,135713,12968,2002-10-28
3,88df75f6a3d343b6,21eddc7dd0ac48a8,Germany,Intern,120276,11267,2014-04-06
4,a0e857607c0e4167,e60b849c20874ba9,France,Full-time,76601,12479,2014-06-05
...,...,...,...,...,...,...,...
995,74be33a6fdd54f18,93472f46e1ba4d79,Germany,Intern,120392,12377,2006-02-01
996,149dfd1141344ea9,71d199d9b2c7444a,France,Full-time,90827,14020,2011-05-03
997,815476138e394933,1a1c283f932640ed,India,Part-time,148690,11970,2020-09-21
998,62ed9d935fc54538,21eddc7dd0ac48a8,China,Contract,111106,17970,2005-04-23


In [26]:
# Generar valores de clave externa aleatorios
def generate_foreign_random_values(reference_values, size):
    return np.random.choice(reference_values, size=size, replace=True)

# Extraer dependencias de un dataset
def extract_dependencies(obj):
    dependencies = []
    for col in obj['columns']:
        if col['type'] == 'foreign':
            df_name, _ = col['values'].split('.')
            dependencies.append(df_name)
    return dependencies

def reorder_by_dependencies(configs):
    sorted_configs = []
    remaining_configs = configs.copy()
    
    while remaining_configs:
        for config in remaining_configs:
            dependencies = extract_dependencies(config)
            if all(dep in [c['ds'] for c in sorted_configs] for dep in dependencies):
                sorted_configs.append(config)
                remaining_configs.remove(config)
                
    return sorted_configs

all_dfs = {}

def build_dataframes(conf_list):
    # Reordenar las configuraciones por dependencias
    sorted_conf_list = reorder_by_dependencies(conf_list)
    
    # Crear los DataFrames en el orden correcto
    for config in sorted_conf_list:
        reference_data = {f"{k}.{col}": v[col].tolist() for k, v in all_dfs.items() for col in v.columns}
        df = generate_dataset(config, reference_data)
        df.name = config['ds']
        all_dfs[df.name] = df
        
    # Reordenar según el orden original de conf_list
    return [all_dfs[config['ds']] for config in conf_list]



# Definir la lista de configuraciones
conf_list = [d2, d1, d5, d3, d4]

# Generar los DataFrames en el orden correcto
dataframe_list = build_dataframes(conf_list)

# Imprimir los DataFrames generados
for df in dataframe_list:
    print(f"DataFrame {df.name}:\n", df.head(), "\n")

DataFrame dataset2:
                  id area subarea
0  f16c7c7b44544d5b   TI     SA1
1  7285eeb736b44fe3  FIN     SA2
2  7a40fa125cd44403   HR     SA3
3  1e1d8d95ae024d1f   TI     SA4 

DataFrame dataset1:
   area                id
0   TI  b31df7cf8f0d4057
1  FIN  9c5ca72acdb3429a
2   HR  9138935b5ae04514 

DataFrame dataset5:
                  id       employee_id  country     status  salary  bonus  \
0  6d61ca5e6e8d4c77  0f7b933286d649da      USA  Full-time   58300  13906   
1  4db92bc119f2451e  adcce1559eea47e8   Canada  Part-time   69458  10121   
2  9a123f57bf1b4585  437ea229ad914ce9       UK   Contract   85227  13913   
3  31ba52e0069b498d  c251d54df11c4257  Germany     Intern   87728  19939   
4  c48e2e3a819b4eb0  b1bca7a9225740f9   France  Full-time   98865  10651   

  joining_date  
0   2021-08-19  
1   2023-09-15  
2   2008-06-17  
3   2004-09-22  
4   2002-05-23   

DataFrame dataset3:
                  id           subarea  income   goal
0  059b7c27b51741f0  f16c7c7b4454

# Función para construir los dataframes en el orden correcto
def build_dataframes(conf_list):
    temp_conf_list = reorder_by_dependencies(conf_list.copy())
    df_arr = []

    for df in temp_conf_list:
        df_arr.append( generate_dataset(df))

    # Reordenar considerando el orden original (conf_list)
    return  [j for i in conf_list for j in df_arr if i['ds'] == j.name]

# Definir la lista de configuraciones
conf_list = [d2, d1, d5, d3, d4]

# Generar los dataframes
dataframe_list = build_dataframes(conf_list)

# Imprimir los dataframes generados
for i, df in enumerate(dataframe_list):
    print(f"Dataset {i+1}:")
    print(df.head(), "\n")  # Imprimir las primeras filas de cada dataframe

In [28]:
dataframe_list[2]

,id,employee_id,country,status,salary,bonus,joining_date
0,6d61ca5e6e8d4c77,0f7b933286d649da,USA,Full-time,58300,13906,2021-08-19
1,4db92bc119f2451e,adcce1559eea47e8,Canada,Part-time,69458,10121,2023-09-15
2,9a123f57bf1b4585,437ea229ad914ce9,UK,Contract,85227,13913,2008-06-17
3,31ba52e0069b498d,c251d54df11c4257,Germany,Intern,87728,19939,2004-09-22
4,c48e2e3a819b4eb0,b1bca7a9225740f9,France,Full-time,98865,10651,2002-05-23
...,...,...,...,...,...,...,...
995,71a13acc86c84d2d,0f7b933286d649da,Germany,Intern,103549,13380,2009-07-19
996,feb84a8196564a67,adcce1559eea47e8,France,Full-time,76385,17322,2020-03-13
997,513eb2faf3b44c3d,437ea229ad914ce9,India,Part-time,115169,12937,2006-02-24
998,62efab99326a418b,c251d54df11c4257,China,Contract,95454,11566,2008-12-19


# Requisito 5

In [91]:
simulation_extended = dataframe_list[2]

In [93]:
simulation_extended

,id,employee_id,country,status,salary,bonus,joining_date,n
0,6d61ca5e6e8d4c77,0f7b933286d649da,USA,Full-time,58300,13906,2021-08-19,1
1,4db92bc119f2451e,adcce1559eea47e8,Canada,Part-time,69458,10121,2023-09-15,1
2,9a123f57bf1b4585,437ea229ad914ce9,UK,Contract,85227,13913,2008-06-17,1
3,31ba52e0069b498d,c251d54df11c4257,Germany,Intern,87728,19939,2004-09-22,1
4,c48e2e3a819b4eb0,b1bca7a9225740f9,France,Full-time,98865,10651,2002-05-23,1
...,...,...,...,...,...,...,...,...
995,71a13acc86c84d2d,0f7b933286d649da,Germany,Intern,103549,13380,2009-07-19,1
996,feb84a8196564a67,adcce1559eea47e8,France,Full-time,76385,17322,2020-03-13,1
997,513eb2faf3b44c3d,437ea229ad914ce9,India,Part-time,115169,12937,2006-02-24,1
998,62efab99326a418b,c251d54df11c4257,China,Contract,95454,11566,2008-12-19,1


In [228]:
category_cols = ['country', 'status']
numeric_cols = ['salary', 'bonus']

size = 100000

Analisis mejor combinacion 

In [254]:
def get_categorical_dataset_simulated(simulation_extended, category_cols, size):
      # Verifica la existencia de las columnas categóricas
    for col in category_cols:
        if col not in simulation_extended.columns:
            raise ValueError(f"Column {col} not found in DataFrame")

    # La columna n = 1 servirá para realizar la función de agrupación count en el DataFrame
    simulation_extended['n'] = 1

    # Generamos la agrupación por columnas categóricas con su respectiva función de agregación count
    grouped_combination = simulation_extended.groupby(
        category_cols,
        as_index=False,
        observed=True  # Cambiado a True para incluir solo combinaciones presentes
    ).agg({
        'n': 'count'
    })

    # Renombramos las columnas
    grouped_combination.columns = category_cols + ['count']

    # Calculamos la probabilidad de cada combinación
    total_rows = simulation_extended.shape[0]
    if total_rows <= 0:
        raise ValueError("Total rows in simulation_extended is zero or negative")
    
    grouped_combination['prob'] = grouped_combination['count'] / total_rows

    # Verificamos la suma de las probabilidades y normalizamos si es necesario
    prob_sum = grouped_combination['prob'].sum()
    if not np.isclose(prob_sum, 1):
        grouped_combination['prob'] /= prob_sum

    # Generamos aleatoriamente un arreglo de índices
    rand_index = np.random.choice(
        grouped_combination.index,
        size=size,
        p=grouped_combination['prob']
    )

    # Retornamos los 100k registros aleatorios ya con sus respectivos valores, no como índices nada más
    return grouped_combination.loc[rand_index, category_cols]


# Generar los registros de manera aleatoria para las columnas numericas
def get_numeric_column_simulated( simulated , simulation_extended , category_cols , column_name ):
    # Funcion de agregacion para obtener el min, max, mean y std por cada agrupacion de columnas categoricas
    a1 = simulation_extended.groupby(
        category_cols
        , as_index = False,
        observed=True
    ).agg(
        {
            column_name: ["min","max","mean","std"]
        }
    )

    a1.columns = [c for c in category_cols] + ["Min", "Max", "Mean", "Std"]  # Renombra las columnas de a1

    ColumnSimulated = pd.DataFrame() # Inicializa un nuevo dataframe vacio en este caso
    
    # Recorre los indices de a1
    for i in a1.index:
        # Toma el registro completo del indice i, el registro completo son toda la fila de valores de ese indice
        rs = a1.loc[i]

        # reasigna final_simulation y crea una copia de todos los registros de simulated (que al principio era una copia de final_simulation sin la columna n) que contengan el indice i
        final_simulation = simulated.loc[i].copy()    

        # Genera los valores aleatorios en base a su resumen estadistico de min, max, mean y std
        data = generate_truncated_normal(
            rs["Min"] - 1 if rs["Std"] == 0 else rs["Min"],
            rs["Max"] + 1 if rs["Std"] == 0 else rs["Max"],
            1 if rs["Std"] == 0 else rs["Std"],
            rs["Mean"], 
            final_simulation.shape[0]
        )

        # Agregamos la nueva columna a final_simulation
        final_simulation[column_name] = data

        # Concatenamos el conjunto de registros anteriores con el registro actual recorrido del ciclo
        ColumnSimulated = pd.concat( [ ColumnSimulated , final_simulation  ] )

    return ColumnSimulated.reset_index(drop=True) # Retornamos el resultado final restableciendo los indices

# Analisis mejor combinacion de variables categoricas

In [164]:
print(category_cols)

['country', 'status']


In [232]:
# Añade la columna 'n' a simulation_extended
simulation_extended['n'] = 1

# Agrupa por columnas categóricas y cuenta el número de ocurrencias
grouped_combination = simulation_extended.groupby(
    category_cols,
    as_index=False,
    observed=True
).agg({
    'n': 'count'
})

# Renombra las columnas de grouped_combination
# Asegúrate de que el número de columnas sea correcto
print("Número de columnas en grouped_combination:", len(grouped_combination.columns))

# Renombra las columnas
grouped_combination.columns = category_cols + ['count']

# Calcula las probabilidades
grouped_combination['prob'] = grouped_combination['count'] / len(simulation_extended)

# Imprime el DataFrame final para verificación
grouped_combination

Número de columnas en grouped_combination: 3


,country,status,count,prob
0,Canada,Part-time,125,0.125
1,China,Contract,125,0.125
2,France,Full-time,125,0.125
3,Germany,Intern,125,0.125
4,India,Part-time,125,0.125
5,Japan,Intern,125,0.125
6,UK,Contract,125,0.125
7,USA,Full-time,125,0.125


In [234]:
simulation_extended['n'] = 1
grouped_combination = simulation_extended.groupby(
    category_cols,
    as_index=False,
    observed=True
).agg({
    'n': 'count'
})

grouped_combination.columns =  category_cols + ['count']
grouped_combination['prob'] = grouped_combination['count'] / dataframe_list[4].shape[0]

In [236]:
# area_subarea sera el resultado que devuelve la funcion get_categoricas_dataset_simulated
final_simulation = get_categorical_dataset_simulated(simulation_extended, category_cols, 100000)
final_simulation

,country,status
4,India,Part-time
3,Germany,Intern
7,USA,Full-time
0,Canada,Part-time
4,India,Part-time
...,...,...
6,UK,Contract
3,Germany,Intern
3,Germany,Intern
1,China,Contract


In [238]:
final_simulation['n'] = 1
result = final_simulation.groupby(
    category_cols,
    as_index=False,
    observed=True
).agg({
    'n': 'count'
})

result.columns = category_cols + ['count']
result['prob'] = result['count'] / final_simulation.shape[0]

In [240]:
# Mejor combinacion de variables categoricas desde la primera agrupacion
grouped_combination.sort_values(by='count', ascending=False)

,country,status,count,prob
0,Canada,Part-time,125,25.0
1,China,Contract,125,25.0
2,France,Full-time,125,25.0
3,Germany,Intern,125,25.0
4,India,Part-time,125,25.0
5,Japan,Intern,125,25.0
6,UK,Contract,125,25.0
7,USA,Full-time,125,25.0


In [242]:
# Probabilidades de las mejores combinaciones despues de haber generado los 100k registros
result.sort_values(by='count', ascending=False)

,country,status,count,prob
2,France,Full-time,12632,0.12632
0,Canada,Part-time,12627,0.12627
4,India,Part-time,12551,0.12551
1,China,Contract,12494,0.12494
3,Germany,Intern,12464,0.12464
7,USA,Full-time,12453,0.12453
6,UK,Contract,12395,0.12395
5,Japan,Intern,12384,0.12384


In [244]:
final_simulation

,country,status,n
4,India,Part-time,1
3,Germany,Intern,1
7,USA,Full-time,1
0,Canada,Part-time,1
4,India,Part-time,1
...,...,...,...
6,UK,Contract,1
3,Germany,Intern,1
3,Germany,Intern,1
1,China,Contract,1


In [256]:
# simulated en este caso es una copia de final_simulation excluyendo la variable n que no pertenece al dataframe original
# final_simulation contiene los 100k registros generados aleatoriamente
#simulated = final_simulation.drop(columns='n', axis=1).copy()

# Restablece los indices a una secuencia por default antes: [0, 0, 0, ..., n, n] ahora: [0,1,2,3,...,n]
final_simulation = simulated.sort_index().reset_index(drop=True).copy()

# Recorremos el arreglo de columnas numericas
for nc in numeric_cols:
    # Por cada valor numerico, se generan los demas registros aleatoriamente
    # simulated = copia de los 100k registros generados aleatorios
    # simulation_extended = dataframe al cual se le esta realizando el analisis. En este caso este dataframe se creo desde el inicio con 1k registros
    # category_cols = arreglo con las columnas categoricas
    # nc = columna numerica actual a medida se recorre el ciclo de las columnas numericas
    dfn = get_numeric_column_simulated(simulated, simulation_extended, category_cols, nc) 

    # Combina el dataframe actual de final_simulation con el dataframe que se va generando a medida recorra el ciclo
    # Los combina por sus indices (Tomando en cuenta que los indices son unicos)
    final_simulation = pd.merge(
        final_simulation
        , dfn.loc[:,[nc]]
        , left_index=True
        , right_index=True
    )

In [258]:
final_simulation

,country,status,salary,bonus
0,Canada,Part-time,82235,7401
1,Canada,Part-time,55410,14109
2,Canada,Part-time,82590,5797
3,Canada,Part-time,73261,14605
4,Canada,Part-time,61496,7032
...,...,...,...,...
99995,USA,Full-time,74360,9913
99996,USA,Full-time,49887,8850
99997,USA,Full-time,75586,14397
99998,USA,Full-time,43020,14202
